Note: This license has also been called the "New BSD License" or 
"Modified BSD License". See also the 2-clause BSD License.

Copyright © 2018-2019 - General Electric Company, All Rights Reserved
 
Project: ANSWER, developed with the support of the Defense Advanced 
Research Projects Agency (DARPA) under Agreement  No.  HR00111990006. 
 
Redistribution and use in source and binary forms, with or without 
modification, are permitted provided that the following conditions are met:

 1. Redistributions of source code must retain the above copyright notice, 
     this list of conditions and the following disclaimer.

 2. Redistributions in binary form must reproduce the above copyright notice, 
     this list of conditions and the following disclaimer in the documentation 
     and/or other materials provided with the distribution.

 3. Neither the name of the copyright holder nor the names of its 
    contributors may be used to endorse or promote products derived 
     from this software without specific prior written permission.

 THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" 
 AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE 
 IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE 
 ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE 
 LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR 
 CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF 
 SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS 
 INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN 
 CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) 
 ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF 
 THE POSSIBILITY OF SUCH DAMAGE.

In [19]:
# This code is used to train a BI-LSTM-CRF Sequence Tagger model
from flair.data import Sentence
from flair.models import SequenceTagger

In [53]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher

#main data dir : nasa-website
base_path = 'dataset/nasa-website/annotations/'
data_folder = base_path + 'data/'

columns = {0: 'text', 1: 'ner'}

corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='trainEquations.txt',
                                                              test_file='testEquations.txt')
#                                                              dev_file='train.txt')


2019-01-24 16:11:28,689 Reading data from dataset/nasa-website/annotations/data
2019-01-24 16:11:28,691 Train: dataset/nasa-website/annotations/data/trainEquations.txt
2019-01-24 16:11:28,693 Dev: None
2019-01-24 16:11:28,695 Test: dataset/nasa-website/annotations/data/testEquations.txt


In [54]:
len(corpus.train)
#print(corpus.train[0].to_tagged_string('ner'))

2281

In [55]:
from flair.embeddings import TokenEmbeddings, CharacterEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List

tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

[b'<unk>', b'O', b'B-CONCEPT', b'I-CONCEPT', b'B-EQUATION', b'I-EQUATION', b'O"', b'B-CONCEPT"', b'<START>', b'<STOP>']


In [56]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)


In [57]:
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [58]:
trainer.train('resources/taggers/ner-equations-01-23-2019',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2019-01-24 16:11:51,612 ----------------------------------------------------------------------------------------------------
2019-01-24 16:11:51,614 Evaluation method: MICRO_F1_SCORE
2019-01-24 16:11:51,616 ----------------------------------------------------------------------------------------------------
2019-01-24 16:11:52,525 epoch 1 - iter 0/72 - loss 51.76071167
2019-01-24 16:11:56,704 epoch 1 - iter 7/72 - loss 19.21198165
2019-01-24 16:12:00,577 epoch 1 - iter 14/72 - loss 12.25069197
2019-01-24 16:12:04,549 epoch 1 - iter 21/72 - loss 9.49504805
2019-01-24 16:12:09,277 epoch 1 - iter 28/72 - loss 8.18955206
2019-01-24 16:12:13,796 epoch 1 - iter 35/72 - loss 7.27675532
2019-01-24 16:12:17,765 epoch 1 - iter 42/72 - loss 6.66108679
2019-01-24 16:12:21,490 epoch 1 - iter 49/72 - loss 6.12638412
2019-01-24 16:12:25,619 epoch 1 - iter 56/72 - loss 5.78524303
2019-01-24 16:12:29,677 epoch 1 - iter 63/72 - loss 5.44226931
2019-01-24 16:12:33,853 epoch 1 - iter 70/72 - loss 5.1560499

2019-01-24 16:18:00,409 DEV  : loss 0.65241188 - f-score 0.7487 - acc 0.7487
2019-01-24 16:18:08,060 TEST : loss 0.85083598 - f-score 0.7502 - acc 0.7502
2019-01-24 16:18:10,578 ----------------------------------------------------------------------------------------------------
2019-01-24 16:18:11,189 epoch 8 - iter 0/72 - loss 0.53701812
2019-01-24 16:18:15,317 epoch 8 - iter 7/72 - loss 0.80177581
2019-01-24 16:18:19,579 epoch 8 - iter 14/72 - loss 0.92357865
2019-01-24 16:18:23,467 epoch 8 - iter 21/72 - loss 0.94912175
2019-01-24 16:18:27,701 epoch 8 - iter 28/72 - loss 1.06683644
2019-01-24 16:18:32,394 epoch 8 - iter 35/72 - loss 1.08887915
2019-01-24 16:18:36,619 epoch 8 - iter 42/72 - loss 1.10253578
2019-01-24 16:18:40,726 epoch 8 - iter 49/72 - loss 1.14119890
2019-01-24 16:18:45,056 epoch 8 - iter 56/72 - loss 1.14014819
2019-01-24 16:18:49,365 epoch 8 - iter 63/72 - loss 1.12694796
2019-01-24 16:18:53,448 epoch 8 - iter 70/72 - loss 1.13344307
2019-01-24 16:18:53,733 ------

2019-01-24 16:24:28,239 DEV  : loss 0.48120484 - f-score 0.8651 - acc 0.8651
2019-01-24 16:24:35,129 TEST : loss 0.67114699 - f-score 0.8355 - acc 0.8355
2019-01-24 16:24:35,133 ----------------------------------------------------------------------------------------------------
2019-01-24 16:24:35,707 epoch 15 - iter 0/72 - loss 1.71474981
2019-01-24 16:24:40,029 epoch 15 - iter 7/72 - loss 1.05520684
2019-01-24 16:24:44,245 epoch 15 - iter 14/72 - loss 1.00095525
2019-01-24 16:24:48,824 epoch 15 - iter 21/72 - loss 0.91639448
2019-01-24 16:24:52,865 epoch 15 - iter 28/72 - loss 0.88583007
2019-01-24 16:24:56,738 epoch 15 - iter 35/72 - loss 0.86349336
2019-01-24 16:25:00,842 epoch 15 - iter 42/72 - loss 0.84999016
2019-01-24 16:25:05,214 epoch 15 - iter 49/72 - loss 0.84099651
2019-01-24 16:25:09,638 epoch 15 - iter 56/72 - loss 0.82168842
2019-01-24 16:25:14,116 epoch 15 - iter 63/72 - loss 0.81611875
2019-01-24 16:25:18,394 epoch 15 - iter 70/72 - loss 0.82494338
2019-01-24 16:25:18

2019-01-24 16:30:48,488 EPOCH 21 done: loss 0.6084 - lr 0.1000 - bad epochs 0
2019-01-24 16:30:50,276 DEV  : loss 0.37498823 - f-score 0.8688 - acc 0.8688
2019-01-24 16:30:57,667 TEST : loss 0.54532629 - f-score 0.8619 - acc 0.8619
2019-01-24 16:31:00,216 ----------------------------------------------------------------------------------------------------
2019-01-24 16:31:00,821 epoch 22 - iter 0/72 - loss 0.57034111
2019-01-24 16:31:04,908 epoch 22 - iter 7/72 - loss 0.54216108
2019-01-24 16:31:09,084 epoch 22 - iter 14/72 - loss 0.44900686
2019-01-24 16:31:13,425 epoch 22 - iter 21/72 - loss 0.49883938
2019-01-24 16:31:17,524 epoch 22 - iter 28/72 - loss 0.55052977
2019-01-24 16:31:21,616 epoch 22 - iter 35/72 - loss 0.60759550
2019-01-24 16:31:25,999 epoch 22 - iter 42/72 - loss 0.64680547
2019-01-24 16:31:30,286 epoch 22 - iter 49/72 - loss 0.64208387
2019-01-24 16:31:34,542 epoch 22 - iter 56/72 - loss 0.63211937
2019-01-24 16:31:38,901 epoch 22 - iter 63/72 - loss 0.64969117
2019-

2019-01-24 16:37:01,116 ----------------------------------------------------------------------------------------------------
2019-01-24 16:37:01,118 EPOCH 28 done: loss 0.5227 - lr 0.0500 - bad epochs 0
2019-01-24 16:37:02,523 DEV  : loss 0.30552691 - f-score 0.9091 - acc 0.9091
2019-01-24 16:37:09,530 TEST : loss 0.47797504 - f-score 0.8854 - acc 0.8854
2019-01-24 16:37:12,058 ----------------------------------------------------------------------------------------------------
2019-01-24 16:37:12,657 epoch 29 - iter 0/72 - loss 0.29982907
2019-01-24 16:37:16,708 epoch 29 - iter 7/72 - loss 0.57266298
2019-01-24 16:37:20,568 epoch 29 - iter 14/72 - loss 0.54827596
2019-01-24 16:37:25,098 epoch 29 - iter 21/72 - loss 0.54624105
2019-01-24 16:37:29,128 epoch 29 - iter 28/72 - loss 0.55518501
2019-01-24 16:37:33,460 epoch 29 - iter 35/72 - loss 0.53287935
2019-01-24 16:37:37,825 epoch 29 - iter 42/72 - loss 0.50998004
2019-01-24 16:37:42,415 epoch 29 - iter 49/72 - loss 0.55138865
2019-01-

...

2019-01-24 17:15:02,066 epoch 70 - iter 14/72 - loss 0.39679316
2019-01-24 17:15:06,243 epoch 70 - iter 21/72 - loss 0.36879033
2019-01-24 17:15:10,520 epoch 70 - iter 28/72 - loss 0.33060300
2019-01-24 17:15:15,063 epoch 70 - iter 35/72 - loss 0.31469344
2019-01-24 17:15:19,763 epoch 70 - iter 42/72 - loss 0.31612374
2019-01-24 17:15:24,570 epoch 70 - iter 49/72 - loss 0.33258257
2019-01-24 17:15:29,037 epoch 70 - iter 56/72 - loss 0.32954421
2019-01-24 17:15:33,347 epoch 70 - iter 63/72 - loss 0.33408408
2019-01-24 17:15:37,878 epoch 70 - iter 70/72 - loss 0.34755536
2019-01-24 17:15:38,182 ----------------------------------------------------------------------------------------------------
2019-01-24 17:15:38,184 EPOCH 70 done: loss 0.3466 - lr 0.0063 - bad epochs 3
2019-01-24 17:15:39,684 DEV  : loss 0.24249567 - f-score 0.9231 - acc 0.9231
2019-01-24 17:15:48,059 TEST : loss 0.42113608 - f-score 0.9217 - acc 0.9217
Epoch    69: reducing learning rate of group 0 to 3.1250e-03.
2019-

2019-01-24 17:21:29,955 epoch 77 - iter 0/72 - loss 0.10195108
2019-01-24 17:21:35,041 epoch 77 - iter 7/72 - loss 0.31974821
2019-01-24 17:21:39,493 epoch 77 - iter 14/72 - loss 0.40219005
2019-01-24 17:21:44,077 epoch 77 - iter 21/72 - loss 0.40036717
2019-01-24 17:21:48,938 epoch 77 - iter 28/72 - loss 0.40614637
2019-01-24 17:21:53,647 epoch 77 - iter 35/72 - loss 0.40443556
2019-01-24 17:21:58,201 epoch 77 - iter 42/72 - loss 0.36774668
2019-01-24 17:22:02,627 epoch 77 - iter 49/72 - loss 0.34175107
2019-01-24 17:22:07,490 epoch 77 - iter 56/72 - loss 0.36669382
2019-01-24 17:22:12,768 epoch 77 - iter 63/72 - loss 0.36035304
2019-01-24 17:22:16,959 epoch 77 - iter 70/72 - loss 0.34962307
2019-01-24 17:22:17,203 ----------------------------------------------------------------------------------------------------
2019-01-24 17:22:17,204 EPOCH 77 done: loss 0.3485 - lr 0.0016 - bad epochs 1
2019-01-24 17:22:18,630 DEV  : loss 0.23805480 - f-score 0.9231 - acc 0.9231
2019-01-24 17:22:2

Epoch    82: reducing learning rate of group 0 to 3.9063e-04.
2019-01-24 17:28:02,844 ----------------------------------------------------------------------------------------------------
2019-01-24 17:28:03,436 epoch 84 - iter 0/72 - loss 0.04405427
2019-01-24 17:28:08,011 epoch 84 - iter 7/72 - loss 0.37800818
2019-01-24 17:28:12,159 epoch 84 - iter 14/72 - loss 0.38161844
2019-01-24 17:28:16,274 epoch 84 - iter 21/72 - loss 0.35018046
2019-01-24 17:28:21,055 epoch 84 - iter 28/72 - loss 0.34377812
2019-01-24 17:28:25,920 epoch 84 - iter 35/72 - loss 0.40821773
2019-01-24 17:28:30,839 epoch 84 - iter 42/72 - loss 0.37360732
2019-01-24 17:28:35,725 epoch 84 - iter 49/72 - loss 0.36550734
2019-01-24 17:28:40,380 epoch 84 - iter 56/72 - loss 0.35124270
2019-01-24 17:28:44,774 epoch 84 - iter 63/72 - loss 0.34351734
2019-01-24 17:28:48,929 epoch 84 - iter 70/72 - loss 0.35050511
2019-01-24 17:28:49,246 ---------------------------------------------------------------------------------------

2019-01-24 17:34:31,692 DEV  : loss 0.23700468 - f-score 0.9231 - acc 0.9231
2019-01-24 17:34:39,820 TEST : loss 0.41780066 - f-score 0.9218 - acc 0.9218
2019-01-24 17:34:39,825 ----------------------------------------------------------------------------------------------------
2019-01-24 17:34:40,617 epoch 91 - iter 0/72 - loss 0.20189869
2019-01-24 17:34:45,317 epoch 91 - iter 7/72 - loss 0.28900978
2019-01-24 17:34:49,708 epoch 91 - iter 14/72 - loss 0.26456805
2019-01-24 17:34:54,271 epoch 91 - iter 21/72 - loss 0.25048022
2019-01-24 17:34:59,472 epoch 91 - iter 28/72 - loss 0.30544078
2019-01-24 17:35:03,981 epoch 91 - iter 35/72 - loss 0.31052737
2019-01-24 17:35:08,207 epoch 91 - iter 42/72 - loss 0.34189788
2019-01-24 17:35:12,991 epoch 91 - iter 49/72 - loss 0.34325391
2019-01-24 17:35:17,204 epoch 91 - iter 56/72 - loss 0.33604837
2019-01-24 17:35:21,698 epoch 91 - iter 63/72 - loss 0.34048313
2019-01-24 17:35:26,410 epoch 91 - iter 70/72 - loss 0.32573955
2019-01-24 17:35:26

{'test_score': 0.9218,
 'dev_score_history': [0.3151,
  0.6178,
  0.6339,
  0.6559,
  0.5061,
  0.6989,
  0.7487,
  0.6989,
  0.7873,
  0.7879,
  0.8465,
  0.8558,
  0.7961,
  0.8651,
  0.8679,
  0.8531,
  0.8585,
  0.8679,
  0.8785,
  0.8207,
  0.8688,
  0.8879,
  0.8663,
  0.8818,
  0.9018,
  0.9041,
  0.8908,
  0.9091,
  0.9099,
  0.8889,
  0.9099,
  0.9196,
  0.9091,
  0.9041,
  0.914,
  0.885,
  0.9099,
  0.914,
  0.914,
  0.9099,
  0.9041,
  0.9091,
  0.9099,
  0.9058,
  0.914,
  0.914,
  0.9091,
  0.914,
  0.914,
  0.914,
  0.914,
  0.9231,
  0.914,
  0.9133,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231,
  0.9231],
 'train_loss_his

In [77]:
model = SequenceTagger.load_from_file('resources/taggers/ner-01-23-2019/final-model.pt')
sentence = Sentence('The momentum is defined to be the mass of an object m times its velocity.')
model.predict(sentence)
print(sentence.to_tagged_string())


The momentum is defined to be the mass <B-CONCEPT> of an object m times its velocity.
